# Análisis de Lesiones Cutáneas para la Detección de Melanoma
## INF395 Introducción a las Redes Neuronales and Deep Learning
- Estudiante: Alessandro Bruno Cintolesi Rodríguez
- ROL: 202173541-0

## 1) Libraries and Imports

In [ ]:
# === Standard Library ===
import os									# Filesystem and environment utilities
import re									# Regular expressions
import random								# Random sampling and shuffling
from pathlib import Path					# Object-oriented filesystem paths
from timeit import default_timer as timer	# Benchmarking and timing

# === Third-party (General / Utility) ===
import numpy as np							# Numerical computing
import pandas as pd							# Data manipulation
import requests								# HTTP requests
from PIL import Image						# Image loading and basic processing
import matplotlib.pyplot as plt				# Plotting and visualization

# === PyTorch Core ===
import torch
from torch import nn						# Neural network modules
import torch.nn.functional as F				# Functional API (activations, losses)
from torch.utils.data import DataLoader, WeightedRandomSampler

# === TorchVision ===
from torchvision import datasets			# Datasets like ImageFolder
from torchvision import transforms			# Image transformations
from torchvision import models              # Models (ResNet-18)

# === Scikit-learn Metrics & Utilities ===
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score

# === Progress & Logging ===
from tqdm.auto import tqdm					# Progress bar (notebook-friendly)

# === Target class order expected by external evaluation (e.g., Kaggle submission) ===
_TARGET_LABELS = ["melanoma", "nevus", "seborrheic_keratosis"]

# === Set model variables ===
MODEL_NAME = "pretrained_resnet18"
NUM_EPOCHS = 30
PRETRAINED_RESTNET18 = True

## 2) Setting up random seed and device

In [ ]:
# Setup random seed
torch.manual_seed(42)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Using device:", device)
if device.type == "cuda":
	print("GPU:", torch.cuda.get_device_name(0))

## 3) Definition of recurrent functions

### 3.1) Function that prints the difference between the start time and the end time of the calculation.

In [ ]:
def print_train_time(start: float, end: float, device: torch.device = None):
	"""Prints difference between start and end time.

	Args:
		start (float): Start time of computation (preferred in timeit format).
		end (float): End time of computation.
		device ([type], optional): Device that compute is running on. Defaults to None.

	Returns:
		float: time between start and end in seconds (higher is longer).
	"""
	total_time = end - start
	print(f"Train time on {device}: {total_time:.3f} seconds")
	return total_time

### 3.2) Function that calculates the accuracy between thruth labels and predictions.

In [ ]:
def accuracy_fn(y_true, y_pred):
	"""Calculates accuracy between truth labels and predictions.

	Args:
		y_true (torch.Tensor): Truth labels for predictions.
		y_pred (torch.Tensor): Predictions to be compared to predictions.

	Returns:
		[torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
	"""
	correct = torch.eq(y_true, y_pred).sum().item()
	acc = (correct / len(y_pred)) * 100
	return acc

### 3.3) Function that evaluates a model on a given Dataloader.

In [ ]:
def eval_model(
	model: torch.nn.Module,
	data_loader: torch.utils.data.DataLoader,
	loss_fn: torch.nn.Module,
	accuracy_fn,
	device: torch.device = device
):
	"""
	Evaluates a model on a given DataLoader and computes loss, accuracy, and weighted ROC-AUC.

	Args:
		model (torch.nn.Module):
			A PyTorch model to be evaluated.
		data_loader (torch.utils.data.DataLoader):
			DataLoader containing the evaluation dataset.
		loss_fn (torch.nn.Module):
			Loss function used to compute model error.
		accuracy_fn (callable):
			Function to compute accuracy between predictions and true labels.
		device (torch.device, optional):
			Device on which to perform the evaluation (CPU or GPU).
			Defaults to the global `device`.

	Returns:
		dict:
			A dictionary containing the following metrics:
			- "model_name": Name of the evaluated model class.
			- "model_loss": Average loss computed over the data_loader.
			- "model_acc": Average accuracy.
			- "roc_auc_weighted": Weighted ROC-AUC score, useful for imbalanced datasets.
	"""
	loss, acc = 0.0, 0.0
	true_labels, all_probs = [], []

	# Move model once
	model.to(device)
	model.eval()

	with torch.inference_mode():
		for X, y in data_loader:
			# Send batch to device
			X = X.to(device, non_blocking=True)
			y = y.to(device, non_blocking=True)

			# 1) Forward pass
			y_pred = model(X)

			# 2) Base metrics
			loss += float(loss_fn(y_pred, y).item())
			acc  += float(accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1)))

			# 3) Collect probabilities for ROC-AUC calculation (stored on CPU)
			if y_pred.ndim == 2 and y_pred.shape[1] > 1:
				# Multiclass (or binary with two logits)
				probs = F.softmax(y_pred, dim=1).detach().cpu()
				all_probs.append(probs)
			else:
				# Binary with single logit
				pos_scores = torch.sigmoid(y_pred).detach().cpu().squeeze(1)
				all_probs.append(pos_scores)

			true_labels.append(y.detach().cpu())

	# Compute average across batches
	loss /= len(data_loader)
	acc /= len(data_loader)

	# Prepare for sklearn
	y_true   = torch.cat(true_labels).numpy()
	prob_blob = torch.cat(all_probs)

	# Weighted ROC-AUC
	try:
		if prob_blob.ndim == 2 and prob_blob.shape[1] > 1:
			roc_auc_weighted = roc_auc_score(
				y_true,
				prob_blob.numpy(),
				multi_class="ovr",
				average="weighted"
			)
		else:
			roc_auc_weighted = roc_auc_score(y_true, prob_blob.numpy())
	except ValueError:
		roc_auc_weighted = float("nan")

	print(
		f"Test  loss: {loss:.5f} | Test accuracy: {acc:.2f}% | ROC-AUC (weighted): {roc_auc_weighted:.4f}\n")

### 3.4) Function that extracts prediction probabilities and corresponding true labels.

In [ ]:
def _append_probs_and_labels(
	logits: torch.Tensor,
	y: torch.Tensor,
	probs_list: list,
	labels_list: list
):
	"""
	Extracts prediction probabilities and corresponding true labels for ROC-AUC computation.

	- For multiclass outputs (logits with shape [N, C], C > 1), applies softmax to get
	  class probabilities.
	- For binary outputs (logit with shape [N] or [N, 1]), applies sigmoid to obtain
	  probability of the positive class.
	- Moves tensors to CPU to avoid unnecessary GPU memory usage.

	Args:
		logits (torch.Tensor):
			Raw model outputs before activation.
		y (torch.Tensor):
			True labels associated with the batch.
		probs_list (list):
			List to store detached probability tensors (CPU).
		labels_list (list):
			List to store detached ground truth labels (CPU).

	Returns:
		None. Modifies probs_list and labels_list in-place.
	"""
	with torch.no_grad():
		if logits.ndim == 2 and logits.shape[1] > 1:
			# Multiclass case -> softmax over classes
			probs = F.softmax(logits, dim=1).detach().cpu()
			probs_list.append(probs)
		else:
			# Binary case -> sigmoid score for positive class
			pos_scores = torch.sigmoid(logits).detach().cpu().squeeze(1)
			probs_list.append(pos_scores)

		labels_list.append(y.detach().cpu())

### 3.5) Function that performs one training epoch over a Dataloader.

In [ ]:
def train_step(
	model: torch.nn.Module,
	data_loader: torch.utils.data.DataLoader,
	loss_fn: torch.nn.Module,
	optimizer: torch.optim.Optimizer,
	accuracy_fn,
	device: torch.device = device,
):
	"""
	Performs one training epoch over a DataLoader and computes average loss, accuracy,
	and weighted ROC-AUC (using CPU for the AUC calculation). Moves data/model to the
	selected device (CPU/GPU) and applies a standard forward/backward/step routine.

	Args:
		model (torch.nn.Module):
			PyTorch model to be trained for one epoch.
		data_loader (torch.utils.data.DataLoader):
			DataLoader providing (inputs, targets) training batches.
		loss_fn (torch.nn.Module):
			Loss function used to compute the optimization objective.
		optimizer (torch.optim.Optimizer):
			Optimizer responsible for updating model parameters.
		accuracy_fn (callable):
			Function that computes accuracy from (y_true, y_pred_labels).
		device (torch.device, optional):
			Device on which to run training (CPU or GPU). Defaults to global `device`.

	Returns:
		tuple[float, float, float]:
			- train_loss (float): Mean loss over the epoch.
			- train_acc (float): Mean accuracy over the epoch.
			- train_auc_weighted (float): Weighted ROC-AUC over the epoch (NaN if undefined).
	"""
	train_loss, train_acc = 0.0, 0.0
	true_labels, all_probs = [], []

	# Move model once
	model.to(device)
	model.train()

	for batch, (X, y) in enumerate(data_loader):
		# Move inputs to device
		X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)

		# 1) Forward pass
		y_pred = model(X)

		# 2) Compute loss + accuracy
		loss = loss_fn(y_pred, y)
		train_loss += float(loss.item())
		train_acc  += float(accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1)))

		# 3) Zero gradients
		optimizer.zero_grad(set_to_none=True)

		# 4) Backward pass
		loss.backward()

		# 5) Optimizer update
		optimizer.step()

		# 6) Accumulate predictions for AUC
		_append_probs_and_labels(y_pred, y, all_probs, true_labels)

		# Progress log
		if batch % 8 == 0:
			print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples")

	# Epoch averaging
	train_loss /= len(data_loader)
	train_acc  /= len(data_loader)

	# ROC-AUC Weighted (CPU fallback)
	try:
		y_true = torch.cat(true_labels).numpy()
		prob_blob = torch.cat(all_probs)

		if prob_blob.ndim == 2 and prob_blob.shape[1] > 1:
			# Multiclass weighted ROC-AUC
			train_auc_weighted = roc_auc_score(
				y_true,
				prob_blob.numpy(),
				multi_class="ovr",
				average="weighted"
			)
		else:
			# Binary ROC-AUC
			train_auc_weighted = roc_auc_score(y_true, prob_blob.numpy())
	except ValueError:
		train_auc_weighted = float("nan")

	print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}% | ROC-AUC (weighted): {train_auc_weighted:.4f}")


### 3.6) Function that evaluates a model on a validation/test Dataloader.

In [ ]:
def test_step(
	data_loader: torch.utils.data.DataLoader,
	model: torch.nn.Module,
	loss_fn: torch.nn.Module,
	accuracy_fn,
	device: torch.device = device
):
	"""
	Evaluates a model on a validation or test DataLoader and computes average loss,
	accuracy, and weighted ROC-AUC. Runs entirely in inference mode and moves
	tensors to the selected device (CPU/GPU) for efficiency.

	Args:
		data_loader (torch.utils.data.DataLoader):
			DataLoader providing validation/test batches.
		model (torch.nn.Module):
			Model to be evaluated.
		loss_fn (torch.nn.Module):
			Loss function used to evaluate prediction error.
		accuracy_fn (callable):
			Function that computes accuracy from (y_true, y_pred_labels).
		device (torch.device, optional):
			Device on which evaluation runs. Defaults to global `device`.

	Returns:
		tuple[float, float, float]:
			- test_loss (float): Mean loss over the evaluation DataLoader.
			- test_acc (float): Mean accuracy.
			- roc_auc_weighted (float): Weighted ROC-AUC score (NaN if undefined).
	"""
	test_loss, test_acc = 0.0, 0.0
	true_labels, all_probs = [], []

	# Move model to device
	model.to(device)
	model.eval()

	with torch.inference_mode():
		for X, y in data_loader:
			# Move batch to device
			X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)

			# 1) Forward pass
			test_pred = model(X)

			# 2) Compute base metrics
			test_loss += float(loss_fn(test_pred, y).item())
			test_acc  += float(accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1)))

			# 3) Accumulate probabilities and labels for ROC-AUC (stored on CPU)
			_append_probs_and_labels(test_pred, y, all_probs, true_labels)

	# Average across batches
	test_loss /= len(data_loader)
	test_acc  /= len(data_loader)

	# ROC-AUC Weighted (CPU-based computation)
	try:
		y_true = torch.cat(true_labels).numpy()
		prob_blob = torch.cat(all_probs)

		if prob_blob.ndim == 2 and prob_blob.shape[1] > 1:
			roc_auc_weighted = roc_auc_score(
				y_true,
				prob_blob.numpy(),
				multi_class="ovr",
				average="weighted"
			)
		else:
			roc_auc_weighted = roc_auc_score(y_true, prob_blob.numpy())
	except ValueError:
		roc_auc_weighted = float("nan")

	print(f"Test  loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}% | ROC-AUC (weighted): {roc_auc_weighted:.4f}")

### 3.7) Function that computes per-channel mean and standar deviation from a training dataset.

In [ ]:
@torch.no_grad()
def compute_mean_std_from_path(
	train_dir,
	img_size=(227, 227),
	batch_size=64,
	num_workers=4,
	use_gpu=True
):
	"""
	Computes per-channel mean and standard deviation from a training dataset located at train_dir.
	A temporary ImageFolder is created using a minimal transform (Resize -> ToTensor) without
	any normalization or augmentation to ensure raw pixel distribution is measured correctly.

	Args:
		train_dir (str or Path):
			Path to the training image directory (must be compatible with ImageFolder structure).
		img_size (tuple[int, int], optional):
			Image size to which all images will be resized before computing statistics.
			Defaults to (227, 227), matching your current training pipeline.
		batch_size (int, optional):
			Batch size used when loading images to calculate statistics.
		num_workers (int, optional):
			Number of subprocesses to use for data loading (CPU threads).
		use_gpu (bool, optional):
			If True and a CUDA GPU is available, accumulation is performed on GPU memory
			for faster computation. Final values are moved back to CPU.

	Returns:
		tuple[list[float], list[float]]:
			- mean: List of channel-wise means (e.g., [0.612, 0.448, 0.391])
			- std:  List of channel-wise standard deviations
	"""
	# Temporary dataset for statistics only (no Normalize, no augmentations)
	stats_tf = transforms.Compose([
		transforms.Resize(img_size),
		transforms.ToTensor()
	])
	tmp_ds = datasets.ImageFolder(root=train_dir, transform=stats_tf)

	device = torch.device("cuda" if (use_gpu and torch.cuda.is_available()) else "cpu")
	loader = DataLoader(
		tmp_ds,
		batch_size=batch_size,
		shuffle=False,
		num_workers=num_workers if torch.cuda.is_available() else 0,
		pin_memory=torch.cuda.is_available()
	)

	sum_c = None
	sum2_c = None
	n_pix_total = 0

	for imgs, *_ in loader:
		# imgs: [N, C, H, W] in [0, 1]
		if device.type == "cuda":
			imgs = imgs.to(device, non_blocking=True)

		N, C, H, W = imgs.shape
		pix = N * H * W

		if sum_c is None:
			sum_c = torch.zeros(C, device=device, dtype=torch.float64)
			sum2_c = torch.zeros(C, device=device, dtype=torch.float64)

		sum_c += imgs.sum(dim=(0, 2, 3), dtype=torch.float64)
		sum2_c += (imgs * imgs).sum(dim=(0, 2, 3), dtype=torch.float64)
		n_pix_total += pix

	# Finalize mean/std and return CPU float lists
	mean = (sum_c / n_pix_total).to("cpu").float()
	var  = (sum2_c / n_pix_total) - (sum_c / n_pix_total) ** 2
	var  = torch.clamp(var, min=0.0)  # Avoid small negative values due to precision
	std  = torch.sqrt(var).to("cpu").float()

	return mean.tolist(), std.tolist()


### 3.8) Function that plots a set of randomly selected images before and after applying a given transform.

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=42):
	"""
	Plots a set of randomly selected images before and after applying a given transform.

	Args:
		image_paths (list[pathlib.Path] or list[str]):
			List of file paths to images that will be sampled and visualized.
		transform (callable or torchvision.transforms):
			Transform to be applied to each image (must return a tensor in [C, H, W] format).
		n (int, optional):
			Number of images to randomly sample and visualize. Defaults to 3.
		seed (int, optional):
			Random seed to ensure reproducible sampling. Defaults to 42.

	Returns:
		None. Displays Matplotlib figures showing original vs transformed images.
	"""
	random.seed(seed)
	random_image_paths = random.sample(image_paths, k=n)

	for image_path in random_image_paths:
		with Image.open(image_path) as f:
			fig, ax = plt.subplots(1, 2)

			# Original image
			ax[0].imshow(f)
			ax[0].set_title(f"Original \nSize: {f.size}")
			ax[0].axis("off")

			# Apply transform and permute for matplotlib compatibility
			transformed_image = transform(f).permute(1, 2, 0)
			ax[1].imshow(transformed_image)
			ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
			ax[1].axis("off")

			fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)


### 3.9) Function that normalizes a label string.

In [ ]:
def _clean_label(s: str) -> str:
	"""
	Normalizes a label string by converting to lowercase, replacing spaces/hyphens
	with underscores, and stripping unsupported characters.

	Args:
		s (str): Raw label string.

	Returns:
		str: Clean normalized label string.
	"""
	return re.sub(r"[^a-z0-9_]+", "", s.strip().lower().replace(" ", "_").replace("-", "_"))

### 3.10) Function that extracts the base filename from a file path.

In [ ]:
def _extract_id_from_path(path: str) -> str:
	"""
	Extracts the base filename (without extension) from a file path.

	Example:
		/a/b/ISIC_0001.jpg -> "ISIC_0001"

	Args:
		path (str): File path to extract the ID from.

	Returns:
		str: File identifier without extension.
	"""
	return os.path.splitext(os.path.basename(path))[0]

### 3.11) Function that maps model output class indices to a fixed desired order.

In [ ]:
def _prob_reorder_indices(dataset, n_classes):
	"""
	Maps model output class indices to a fixed desired order defined in _TARGET_LABELS.
	If the dataset exposes class_to_idx, this function attempts to align the output
	probabilities with the expected label order (useful for generating CSV submissions).

	Args:
		dataset: Dataset object with optional class_to_idx mapping.
		n_classes (int): Number of output classes from the model.

	Returns:
		list[int]: Index mapping to reorder probability outputs. Identity map if matching fails.
	"""
	idx_map = list(range(n_classes))  # default identity map
	
	class_to_idx = getattr(dataset, "class_to_idx", None)
	if isinstance(class_to_idx, dict) and len(class_to_idx) == n_classes:
		idx_to_class = {v: k for k, v in class_to_idx.items()}
		derived = []
		for lab in _TARGET_LABELS:
			lab_clean = _clean_label(lab)
			found_idx = None
			for i in range(n_classes):
				ds_lab = _clean_label(str(idx_to_class.get(i, "")))
				if ds_lab == lab_clean:
					found_idx = i
					break
			if found_idx is None:
				# Fallback to identity if any target label is not matched
				return idx_map
			derived.append(found_idx)
		return derived

	return idx_map

### 3.12) Function that attempts to infer image identifiers.

In [ ]:
def _infer_id_sequence(test_loader):
	"""
	Attempts to infer image identifiers by inspecting dataset attributes such as
	`samples`, `imgs`, or custom path lists. This works well when shuffle=False.

	Args:
		test_loader (torch.utils.data.DataLoader): Loader for test/inference dataset.

	Returns:
		list[str] or None: Ordered list of image IDs if paths are accessible, else None.
	"""
	ds = getattr(test_loader, "dataset", None)
	for attr in ("samples", "imgs"):  # list of (path, label)
		if hasattr(ds, attr):
			pairs = getattr(ds, attr)
			return [_extract_id_from_path(p) for p, _ in pairs]
	for attr in ("image_paths", "images", "files", "paths"):
		if hasattr(ds, attr):
			paths = getattr(ds, attr)
			if isinstance(paths, (list, tuple)) and isinstance(paths[0], str):
				return [_extract_id_from_path(p) for p in paths]
	return None

## 4) Data Preparation

In [ ]:
# Define data directory
image_path = Path("skin-lesions/")
print(f"Image path exists: {image_path.exists()}")

### 4.1) Function that walks recursively through a directory and prints a summary of its contents.

In [ ]:
def walk_through_dir(dir_path):
	"""
	Walks recursively through a directory and prints a summary of its contents.

	Args:
		dir_path (str or pathlib.Path):
			Target directory to inspect.

	Returns:
		None. Prints:
			- Number of subdirectories inside each folder.
			- Number of files (images) found in each folder.
			- Full path of each folder visited.
	"""
	for dirpath, dirnames, filenames in os.walk(dir_path):
		print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


In [ ]:
# Cheking the content of our image path
walk_through_dir(image_path)

In [ ]:
# Define train, test and validation directories
train_dir = image_path / "train"
test_dir = image_path / "test"
val_dir = image_path / "valid"

train_dir, test_dir, val_dir

## 5) Visualizing our data

In [ ]:
# Get all image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))

# Select a random image path
random_image_path = random.choice(image_path_list)

# Get the class name from the image path
class_name = random_image_path.parent.name

# Open the image
img = Image.open(random_image_path)

# Print out some information about the image
print(f"Image path: {random_image_path}")
print(f"Class name: {class_name}")
print(f"Image size: {img.size}")
print(f"Image format: {img.format}")

In [ ]:
# Show the image
img.show()

## 6) Transforming Data

In [ ]:
# Calculating the mean and std of our training dataset
if PRETRAINED_RESTNET18:
	DATASET_MEAN = [0.485, 0.456, 0.406]
	DATASET_STD = [0.229, 0.224, 0.225]
else:
	DATASET_MEAN, DATASET_STD = compute_mean_std_from_path(
		train_dir,
		img_size=(227, 227),
		batch_size=64,
		num_workers=4,
		use_gpu=True
	)

In [ ]:
# Printing out the mean and std
print("mean (training dataset):", DATASET_MEAN)
print("std (training dataset):", DATASET_STD)

### 6.1) Defining data transformers for training and evaluation data.

In [ ]:
# Training data transformer
train_data_transform = transforms.Compose([
	transforms.Resize(size=(227, 227)),
	transforms.TrivialAugmentWide(num_magnitude_bins=31),
	transforms.ToTensor(),
	transforms.Normalize(mean=DATASET_MEAN, std=DATASET_STD)
])

# Training data transformer
eval_data_transform = transforms.Compose([
	transforms.Resize(size=(227, 227)),
	transforms.ToTensor(),
	transforms.Normalize(mean=DATASET_MEAN, std=DATASET_STD)
])

In [ ]:
# Plotting random images to show the images before and after appliying the transformer
plot_transformed_images(image_path_list, train_data_transform, n=3, seed=42)

## 7) Loading our images into datasets

In [ ]:
# Create train, test and validation datasets
train_data = datasets.ImageFolder(
	root=train_dir,
	transform=train_data_transform
)

test_data = datasets.ImageFolder(
	root=test_dir,
	transform=eval_data_transform
)

val_data = datasets.ImageFolder(
	root=val_dir,
	transform=eval_data_transform
)

In [ ]:
# Print out some information about the datasets
print(f"- Train data: {train_data}\n\n- Test data: {test_data}\n\n- Validation data: {val_data}")

In [ ]:
# Print out some information about the datasets
print(f"Number of classes: {len(train_data.classes)}")
print(f"Class names: {train_data.classes}")
print(f"Class dictionary: {train_data.class_to_idx}")
print(f"Number of training images: {len(train_data)}")
print(f"Number of test images: {len(test_data)}")
print(f"Number of validation images: {len(val_data)}")

In [ ]:
# Print out some information about an image in our training dataset
img, label = train_data[0][0], train_data[0][1]
print(f"Image shape: {img.shape} | Image datatype: {img.dtype} | Label: {label} | Label datatype: {type(label)}")
print(f"Image Tensor:\n{img}")

## 8) View an Image from our Dataset

In [ ]:
# Change image shape for plotting
img_prmt = img.permute(1, 2, 0)

# Print out image shape and datatype
print(f"Image shape: {img.shape}")
print(f"Permuted image shape: {img_prmt.shape}")

# Plot the image
plt.imshow(img_prmt)
plt.title(f"Label: {label}, Class name: {train_data.classes[label]}")
plt.axis("off")
plt.show()

## 9) Loading our data into DataLoaders

In [ ]:
# Use Sampler to handle unbalanced classes
targets = np.array(train_data.targets)
class_sample_count = np.bincount(targets)
weights = 1.0 / class_sample_count
sample_weights = weights[targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoaders for training, testing, and validation datasets
train_loader = DataLoader(
	train_data,
	batch_size=64,
	num_workers=4 if torch.cuda.is_available() else 0,
	pin_memory=torch.cuda.is_available(), 
	prefetch_factor=2 if torch.cuda.is_available() else None, 
	sampler=sampler,
)

test_loader = DataLoader(
	test_data,
	batch_size=64,
	num_workers=4 if torch.cuda.is_available() else 0,
	pin_memory=torch.cuda.is_available(),  
	prefetch_factor=2 if torch.cuda.is_available() else None, 
	shuffle=False
)

val_loader = DataLoader(
	val_data,
	batch_size=64,
	num_workers=4 if torch.cuda.is_available() else 0,
	pin_memory=torch.cuda.is_available(),  
	prefetch_factor=2 if torch.cuda.is_available() else None, 
	shuffle=False
)

In [ ]:
# Get a batch of data
train_features_batch, train_labels_batch = next(iter(train_loader))

# Print out some information about the batch
print(f"Features batch shape: {train_features_batch.shape} | Features datatype: {train_features_batch.dtype}")
print(f"Label batch shape: {train_labels_batch.shape} | Label datatype: {train_labels_batch.dtype}")

In [ ]:
# Pick a random image from the batch
idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[idx], train_labels_batch[idx]

# Change image shape for plotting
img_prmt = img.permute(1, 2, 0)

# Plot a random image from the batch
plt.imshow(img_prmt)
plt.title(f"Label: {label}, Class name: {train_data.classes[label]}")
plt.axis("off")
plt.show()

## 10) Defining our CNN Model

### 10.1) Trying out the AlexNet architecture.

In [ ]:
# Defining the model using the AlexNet architecture
class AlexNet(nn.Module):
	def __init__(self, input_channels: int, num_classes: int):
		super().__init__()
		# Bloques convolucionales con BatchNorm; AdaptiveAvgPool2d(6,6) para tamaño robusto
		self.features = nn.Sequential(
			# Conv1
			nn.Conv2d(input_channels, 96, kernel_size=11, stride=4, padding=2),
			nn.ReLU(inplace=True),
			nn.BatchNorm2d(96),
			nn.MaxPool2d(kernel_size=3, stride=2),

			# Conv2
			nn.Conv2d(96, 256, kernel_size=5, padding=2),
			nn.ReLU(inplace=True),
			nn.BatchNorm2d(256),
			nn.MaxPool2d(kernel_size=3, stride=2),

			# Conv3
			nn.Conv2d(256, 384, kernel_size=3, padding=1),
			nn.ReLU(inplace=True),
			nn.BatchNorm2d(384),

			# Conv4
			nn.Conv2d(384, 384, kernel_size=3, padding=1),
			nn.ReLU(inplace=True),
			nn.BatchNorm2d(384),

			# Conv5
			nn.Conv2d(384, 256, kernel_size=3, padding=1),
			nn.ReLU(inplace=True),
			nn.BatchNorm2d(256),
			nn.MaxPool2d(kernel_size=3, stride=2),

			# Asegura 6x6 sin depender del tamaño de entrada exacto
			nn.AdaptiveAvgPool2d((6, 6))
		)

		self.classifier = nn.Sequential(
			nn.Flatten(),
			nn.Dropout(p=0.5),
			nn.Linear(256 * 6 * 6, 4096),
			nn.ReLU(inplace=True),
			nn.Dropout(p=0.5),
			nn.Linear(4096, 4096),
			nn.ReLU(inplace=True),
			nn.Linear(4096, num_classes)
		)

		self._init_weights()

	def _init_weights(self):
		# Inicialización Kaiming adecuada para ReLU
		for m in self.modules():
			if isinstance(m, nn.Conv2d):
				nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
				if m.bias is not None:
					nn.init.zeros_(m.bias)
			elif isinstance(m, nn.Linear):
				nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
				if m.bias is not None:
					nn.init.zeros_(m.bias)
			elif isinstance(m, nn.BatchNorm2d):
				nn.init.ones_(m.weight)
				nn.init.zeros_(m.bias)

	def forward(self, x):
		x = self.features(x)
		x = self.classifier(x)
		return x

In [ ]:
# Instantiate our AlexNet model
alexnet_model = AlexNet(
	input_channels=3,
	num_classes=len(train_data.classes),
)

# Send to device
model = alexnet_model.to(device=device)

### 10.2a) Trying out the ResNet-18 architecture.

In [ ]:
# Instantiate our ResNet-18 model
resnet18_model = models.resnet18(
	weights=None,
	num_classes=len(train_data.classes)
)

# Replace final classification layer
resnet18_model.fc = nn.Linear(
	resnet18_model.fc.in_features,
	len(train_data.classes)
)

# Send to device
model = resnet18_model.to(device=device)

### 10.2b) Trying out the pretrained ResNet-18 architecture.

In [ ]:
# Instantiate our pretrained ResNet-18 model
pretrained_resnet18_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Replace final classification layer
pretrained_resnet18_model.fc = nn.Linear(
	pretrained_resnet18_model.fc.in_features,
	len(train_data.classes)
)

# Send to device
model = pretrained_resnet18_model.to(device=device)

### 10.3) Calculating class weights to manage unbalanced classes and setting up our loss and optimizer.

In [ ]:
class FocalLoss(nn.Module):
	def __init__(self, alpha=None, gamma=2.0, reduction="mean"):
		super().__init__()
		self.alpha = alpha  # tensor of class weights or None
		self.gamma = gamma
		self.reduction = reduction

	def forward(self, logits, targets):
		ce = F.cross_entropy(logits, targets, weight=self.alpha, reduction="none")
		pt = torch.softmax(logits, dim=1).gather(1, targets.unsqueeze(1)).squeeze(1).clamp_min(1e-8)
		focal = (1 - pt) ** self.gamma * ce
		if self.reduction == "mean":
			return focal.mean()
		elif self.reduction == "sum":
			return focal.sum()
		return focal

In [ ]:
# Calculate class weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_data.targets), y=train_data.targets)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device=device)

# Setup loss
# loss_fn = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)
loss_fn = FocalLoss(alpha=class_weights, gamma=1.5) 

# Setup optimizer
if PRETRAINED_RESTNET18:
	# Only optimize parameters that require grad
	trainable_params = filter(lambda p: p.requires_grad, model.parameters())
	optimizer = torch.optim.SGD(params=trainable_params, lr=0.01, momentum=0.9, weight_decay=5e-4)
else:
	optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [ ]:
# Setting up our scheduler
steps_per_epoch = len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
	optimizer=optimizer,
	max_lr=0.03,
	steps_per_epoch=steps_per_epoch,
	epochs=NUM_EPOCHS,
	pct_start=0.1,
	anneal_strategy="cos",
	div_factor=10.0,
	final_div_factor=100.0	
)

### 10.4) Training and evaluating our model using the training and evaluating DataLoaders.

In [ ]:
train_time_start_on_gpu = timer()

epochs = NUM_EPOCHS
for epoch in tqdm(range(epochs)):
	print(f"Epoch: {epoch}\n---------")
	train_step(
		data_loader=train_loader,
		model=model,
		loss_fn=loss_fn,
		optimizer=optimizer,
		accuracy_fn=accuracy_fn)
	test_step(
		data_loader=val_loader,
		model=model,
		loss_fn=loss_fn,
		accuracy_fn=accuracy_fn)
	scheduler.step()

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(
	start=train_time_start_on_gpu,
	end=train_time_end_on_gpu,
	device=device)

### 10.5) Evaluate our model using the testing DataLoader and create the Kaggle CSV.

In [ ]:
eval_model(
	model=model,
	data_loader=test_loader,
	loss_fn=loss_fn,
	accuracy_fn=accuracy_fn
)

In [ ]:
# ---------- Inference & CSV export loop ----------
model.eval()
all_rows = []
n_classes = None

# Attempt to infer global ID order for consistent o]utput
global_ids = _infer_id_sequence(test_loader)
global_cursor = 0

with torch.no_grad():
	for batch in test_loader:
		# Automatically detect whether batch contains images only or also IDs/labels
		if isinstance(batch, (list, tuple)):
			if len(batch) == 3:
				X, y_or_ids, ids_or_none = batch
				if torch.is_tensor(y_or_ids):
					X, ids = X, ids_or_none  # (images, labels, ids)
				else:
					X, ids = X, y_or_ids      # (images, ids, _)
			elif len(batch) == 2:
				a, b = batch
				if torch.is_tensor(b) and torch.is_tensor(a):
					X, ids = a, None           # (images, labels)
				elif torch.is_tensor(a):
					X, ids = a, b              # (images, ids)
				else:
					X, ids = a, b
			else:
				X, ids = batch, None
		else:
			X, ids = batch, None

		# Forward pass on device
		X = X.to(device)
		logits = model(X)

		# Get number of classes once
		if n_classes is None:
			n_classes = logits.shape[-1]

		# Convert logits to probabilities
		probs = torch.softmax(logits, dim=1).cpu().numpy()

		# Resolve IDs
		if ids is not None:
			try:
				batch_ids = [os.path.splitext(os.path.basename(str(i)))[0] for i in ids]
			except Exception:
				batch_ids = [str(i) for i in ids]
		elif global_ids is not None:
			bsz = probs.shape[0]
			batch_ids = global_ids[global_cursor:global_cursor + bsz]
			global_cursor += bsz
		else:
			batch_ids = [f"val_{len(all_rows) + i:06d}" for i in range(probs.shape[0])]

		# Map output indices if class order differs from target submission format
		idx_map = _prob_reorder_indices(getattr(test_loader, "dataset", None), probs.shape[1])

		for img_id, p in zip(batch_ids, probs):
			p_ordered = [float(p[idx_map[i]]) for i in range(len(_TARGET_LABELS))]
			all_rows.append({
				"Id": img_id,
				"melanoma": p_ordered[0],
				"nevus": p_ordered[1],
				"seborrheic_keratosis": p_ordered[2],
			})

# ---------- Create CSV ----------
df_sub = pd.DataFrame(all_rows, columns=["Id"] + _TARGET_LABELS)
df_sub["Id"] = df_sub["Id"].astype(str) + ".jpg"
df_sub.to_csv(f"submission_{MODEL_NAME}.csv", index=False)

print(f"✅ CSV generated: submission.csv | Total rows: {len(df_sub)}")
df_sub.head()